In [1]:
from google.cloud import bigquery
import os
import pandas as pd

In [2]:
# make sure you have your GCP credentials in order to access the database
# ... and remember never to commit these into GitHub!
# more info: https://docs.opensource.observer/docs/integrate/python-notebooks#obtain-a-gcp-service-account-key

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../../gcp_credentials.json"
client = bigquery.Client()

In [3]:
apps = pd.read_csv('data/applications_reviewed.csv', index_col=0)

In [4]:
DATE_START = "'2024-03-01'"
DATE_END = "'2024-09-01'"
PROJECT_NAME = 'opensource-observer'
DATAPATH = 'data/sunny_txn_window.parquet'

In [5]:
query = f"""
    select
      a.*,
      p.username
    from `{PROJECT_NAME}.oso.stg_farcaster__addresses` a
    join `{PROJECT_NAME}.oso.stg_farcaster__profiles` p
     on a.fid = p.farcaster_id
    where p.username is not null
"""
# result = client.query(query)
# fc = result.to_dataframe()
# fc = fc[fc['address'].apply(lambda x: x[:2] == '0x')]
# fc.set_index('address', drop=True, inplace=True)
# fc.to_parquet('data/farcaster.parquet')

fc = pd.read_parquet('data/farcaster.parquet')
fids = fc['fid'].to_dict()
fusers = fc['username'].to_dict()

In [6]:
address_list = set([x.lower() for x in apps['address'].dropna()])
address_list_str = "'" + "','".join(address_list) + "'"

query = f"""
with txns as (
    select
      to_address,from_address,block_timestamp, 'OP Mainnet' as chain
    from `{PROJECT_NAME}.superchain.optimism_transactions`
    where
      block_timestamp between {DATE_START} and {DATE_END}
      and receipt_status = 1
    union all
    select
      to_address,from_address,block_timestamp, 'Base' as chain
    from `{PROJECT_NAME}.superchain.base_transactions`
    where
      block_timestamp between {DATE_START} and {DATE_END}
      and receipt_status = 1
    union all
    select
      to_address,from_address,block_timestamp, 'Zora' as chain
    from `{PROJECT_NAME}.superchain.zora_transactions`
    where
      block_timestamp between {DATE_START} and {DATE_END}
      and receipt_status = 1  
    union all
    select
      to_address,from_address,block_timestamp, 'Mode' as chain
    from `{PROJECT_NAME}.superchain.mode_transactions`
    where
      block_timestamp between {DATE_START} and {DATE_END}
      and receipt_status = 1
    union all
    select
      to_address,from_address,block_timestamp, 'Frax' as chain
    from `{PROJECT_NAME}.superchain.frax_transactions`
    where
      block_timestamp between {DATE_START} and {DATE_END}
      and receipt_status = 1
    union all
    select
      to_address,from_address,block_timestamp, 'Metal' as chain
    from `{PROJECT_NAME}.superchain.metal_transactions`
    where
      block_timestamp between {DATE_START} and {DATE_END}
      and receipt_status = 1  
)
select *
from txns
where
    to_address in ({address_list_str})
    or from_address in ({address_list_str})
"""

# result = client.query(query)
# df = result.to_dataframe()
# df.to_parquet(DATAPATH)

In [7]:
df = pd.read_parquet(DATAPATH)
df['block_timestamp'] = pd.to_datetime(df['block_timestamp'])
df['block_timestamp'] = df['block_timestamp'].dt.tz_convert(None)
df['date'] = df['block_timestamp'].dt.date
df['chain'] = df['chain'].str.upper()
df['from_fid'] = df['from_address'].map(fids)
df['to_fid'] = df['to_address'].map(fids)

In [10]:
DF_90  = df[df['block_timestamp'].between(pd.Timestamp('2024-06-01'), pd.Timestamp(DATE_END))]
DF_180 = df[df['block_timestamp'].between(pd.Timestamp(DATE_START),   pd.Timestamp(DATE_END))]

def transactions(dff):
    return len(dff)

def active_addresses(dff):
    return dff['address'].nunique()

def daily_active_addresses(dff):
    return dff.groupby('date')['address'].nunique().mean()

def active_users(dff):
    return len(
        set(
            dff['to_fid'].dropna().unique()
        ).union(
            set(
                dff['from_fid'].dropna().unique()
            )
        )
    )


MODULES = {
    'transactions': transactions,
    'active_addresses': active_addresses,
    'daily_active_addresses': daily_active_addresses,
    'farcaster_users': active_users
}

In [11]:
def query_contract_metrics(project_data):    
    
    if project_data['address_type'] != 'contract':
        return
    
    uid = project_data['id']
    category = project_data['category']
    chain = project_data['chain'].upper()
    address = project_data['address'].lower()

    metrics = []
    project_info = {'id': uid, 'category': category, 'chain': chain, 'address': address}

    for period in ('90D', '180D'):
        
        dff = DF_90 if period == '90D' else DF_180
        dff = dff[(dff['chain'] == chain) & ((dff['to_address'] == address) | (dff['from_address'] == address))].copy()
        if not len(dff):
            return
        dff['address'] = dff.apply(lambda x: x['from_address'] if x['to_address'] == address else x['to_address'], axis=1)
        for (metric_name, metric_func) in MODULES.items():
            value = metric_func(dff)
            if value:
                metric_info = project_info.copy()
                metric_info.update({
                    'metric_name': '_'.join([metric_name, period]),
                    'metric_value': value
                })
            metrics.append(metric_info)
    return metrics

In [12]:
apps.iloc[0]

id                        0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...
recipient                        0xE4EE538019673501F4B75de5aF5CC073Ec0A1487
time                                                             1724860607
name                                                                     0x
status                                                             approved
profile_name                                                     ewokafloka
profile_url                                 https://warpcast.com/ewokafloka
metadata_name                                                            0x
metadata_website                                            https://0x.org/
project_type                                                            App
category                                                                DEX
address_type                                                       contract
address                          0xdef1c0ded9bec7f1a1670819833240f027b25eff
chain_id    

In [13]:
query_contract_metrics(apps.iloc[0])

[{'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'BASE',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'metric_name': 'transactions_90D',
  'metric_value': 1543115},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'BASE',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'metric_name': 'active_addresses_90D',
  'metric_value': 154685},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'BASE',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'metric_name': 'daily_active_addresses_90D',
  'metric_value': 3898.2608695652175},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'BASE',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'metric_name': 'farcaster_users_90D',
  'metric_value': 9828},
 {'id': 

In [14]:
data = []
for i,app in apps.iterrows():
    result = query_contract_metrics(app)
    prj = str(i).zfill(4)
    if result:
        print('-', prj, "SUCCESS")
        data.extend(result)

- 0000 SUCCESS
- 0008 SUCCESS
- 0018 SUCCESS
- 0022 SUCCESS
- 0026 SUCCESS
- 0027 SUCCESS
- 0034 SUCCESS
- 0045 SUCCESS
- 0060 SUCCESS
- 0062 SUCCESS
- 0063 SUCCESS
- 0064 SUCCESS
- 0065 SUCCESS
- 0084 SUCCESS
- 0085 SUCCESS
- 0095 SUCCESS
- 0098 SUCCESS
- 0099 SUCCESS
- 0100 SUCCESS
- 0102 SUCCESS
- 0103 SUCCESS
- 0108 SUCCESS
- 0124 SUCCESS
- 0125 SUCCESS
- 0127 SUCCESS
- 0171 SUCCESS
- 0177 SUCCESS
- 0195 SUCCESS
- 0198 SUCCESS
- 0199 SUCCESS
- 0200 SUCCESS
- 0202 SUCCESS
- 0206 SUCCESS
- 0208 SUCCESS
- 0220 SUCCESS
- 0222 SUCCESS
- 0223 SUCCESS
- 0224 SUCCESS
- 0228 SUCCESS
- 0230 SUCCESS
- 0235 SUCCESS
- 0237 SUCCESS
- 0243 SUCCESS
- 0253 SUCCESS
- 0262 SUCCESS
- 0265 SUCCESS
- 0280 SUCCESS
- 0281 SUCCESS
- 0292 SUCCESS
- 0293 SUCCESS
- 0303 SUCCESS
- 0310 SUCCESS
- 0311 SUCCESS
- 0346 SUCCESS
- 0357 SUCCESS
- 0360 SUCCESS
- 0364 SUCCESS
- 0367 SUCCESS
- 0374 SUCCESS
- 0378 SUCCESS
- 0381 SUCCESS
- 0383 SUCCESS
- 0391 SUCCESS
- 0397 SUCCESS
- 0398 SUCCESS
- 0404 SUCCESS
- 0415 SUC

In [15]:
df_metrics = pd.DataFrame(data)
df_metrics['metric_value'] = df_metrics['metric_value'].apply(int)
df_metrics

,id,category,chain,address,metric_name,metric_value
0,0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...,DEX,BASE,0xdef1c0ded9bec7f1a1670819833240f027b25eff,transactions_90D,1543115
1,0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...,DEX,BASE,0xdef1c0ded9bec7f1a1670819833240f027b25eff,active_addresses_90D,154685
2,0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...,DEX,BASE,0xdef1c0ded9bec7f1a1670819833240f027b25eff,daily_active_addresses_90D,3898
3,0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...,DEX,BASE,0xdef1c0ded9bec7f1a1670819833240f027b25eff,farcaster_users_90D,9828
4,0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...,DEX,BASE,0xdef1c0ded9bec7f1a1670819833240f027b25eff,transactions_180D,2769982
...,...,...,...,...,...,...
1787,0x6a183a25e900116ff76b76b7853fb8fc0d9eafded731...,Community & Curation,ZORA,0xdb8581ad741fba17b8881f73a513ba74ce357302,farcaster_users_90D,4
1788,0x6a183a25e900116ff76b76b7853fb8fc0d9eafded731...,Community & Curation,ZORA,0xdb8581ad741fba17b8881f73a513ba74ce357302,transactions_180D,26
1789,0x6a183a25e900116ff76b76b7853fb8fc0d9eafded731...,Community & Curation,ZORA,0xdb8581ad741fba17b8881f73a513ba74ce357302,active_addresses_180D,23
1790,0x6a183a25e900116ff76b76b7853fb8fc0d9eafded731...,Community & Curation,ZORA,0xdb8581ad741fba17b8881f73a513ba74ce357302,daily_active_addresses_180D,1


In [16]:
df_metrics.to_csv("data/oso_metrics.csv")

In [19]:
df_metrics.pivot_table(
    index=['id', 'category', 'chain', 'address'],
    columns='metric_name',
    values='metric_value'
).to_csv("data/oso_metrics.csv")